# Práctica 2: Procesamiento del Lenguaje Natural

__Fecha de entrega: 8 de mayo de 2023__

> El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de PLN. La práctica consta de 2 notebooks que se entregarán simultáneamente en la tarea de entrega habilitada en el Campus  Virtual.
>
>Lo más importante en esta práctica no es el código Python, sino el análisis de los datos y modelos que construyas y las explicaciones razonadas de cada una de las decisiones que tomes. __No se valorarán trozos de código o gráficas sin ningún tipo de contexto o explicación__.
>
> Finalmente, recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones).

In [ ]:
RANDOM_STATE = 1234

# Apartado 1: Análisis de sentimientos con word embeddings


__Número de grupo: 1__

__Nombres de los estudiantes: Fernando Isaías Leal Sánchez y Jinqing Cai__

## 1) Carga del conjunto de datos

> El fichero `IMBD_Dataset.csv` contiene opiniones de películas clasificadas en 2 categorías diferentes (positiva/negativa).
>
> Este set de datos se creó utilizando el "IMDB Dataset of 50K Movie Reviews", el cual contiene 50,000 reseñas de películas con un sentimiento positivo o negativo adjunto a ellas.

In [ ]:
# acceso a google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt

%matplotlib inline

Cambiamos la representación del `sentiment`: 1 si es positivo, y 0 si es negativo. La razón es porque la librería `keras` exige que la segunda columna sea de tipo numérico, y dará excepción si es de tipo `String`.

In [ ]:
imbd_file = '/content/drive/MyDrive/IA2/IMDB_Dataset.csv'

df = pd.read_csv(imbd_file)
df['sentiment'] = np.array(df['sentiment'] == 'positive', dtype=int)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


> Muestra un ejemplo de cada clase.

In [ ]:
# Hay dos clases: positivo o negativo
pd.concat(df[df.sentiment == sentiment].head(1) for sentiment in (0, 1))

,review,sentiment
3,Basically there's a family where a little boy ...,0
0,One of the other reviewers has mentioned that ...,1


> Haz un estudio del conjunto de datos. ¿qué palabras aparecen más veces?, ¿tendría sentido normalizar de alguna manera el corpus?

Lo primero, para contar el número de apariciones de una palabra en el conjunto de datos, pasamos a transformar nuestros documentos en bolsas de palabras. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
doc_word_freq = vectorizer.fit_transform(df.review)
doc_word_freq

<50000x101583 sparse matrix of type '<class 'numpy.int64'>'
	with 4434500 stored elements in Compressed Sparse Row format>

`vector_data` es ahora una matriz de $50,000 \times 101,583$ donde para cada review $r$ y cada palabra $i$ nos asocia el número de apariciones de $i$ en $r$

In [ ]:
#print([(i,count) for i in range(vector_data.shape[0]) if (count := vector_data[11,i]) > 0])
r = 11
i = 151
feature_names = vectorizer.get_feature_names_out()
feature_names[i], doc_word_freq[r,i]

('12', 2)

Esto nos dice que hay 2 apariciones de la palabra 151 (que se corresponde con el string `"12"`) en la review 11

In [ ]:
review_11 = df.loc[11].review
palabra_151 = feature_names[151]
review_11.count(palabra_151) == doc_word_freq[r,i]

True

Para sacar la palabra que aparece con más frecuencia, hacemos una **suma de las apariciones de cada palabra** en general, y buscamos aquella 

In [ ]:
def get_top_10_freq_words(vector_data):
    word_freq = np.squeeze(np.asarray(vector_data.sum(axis=0)))
    top_10_freq_words = word_freq.argsort()[::-1][:10]
    return top_10_freq_words, word_freq

In [ ]:
top_10_freq_words, word_freq = get_top_10_freq_words(doc_word_freq)
for top, word in enumerate(top_10_freq_words):
    print(f"Position {top + 1}: {feature_names[word]} with frequency {word_freq[word]}")

Position 1: br with frequency 201951
Position 2: movie with frequency 87971
Position 3: film with frequency 79705
Position 4: like with frequency 40172
Position 5: just with frequency 35184
Position 6: good with frequency 29753
Position 7: time with frequency 25110
Position 8: story with frequency 23119
Position 9: really with frequency 23094
Position 10: bad with frequency 18473


Aparece palabras como `br`, que es la etiqueta de HTML para hacer cambio de línea (normal que aparezca en nuestro dataset, pues es conjunto de reseñas de películas extraidas de una página web).

Por lo tanto, es necesario que hacer un preprocesamiento del dataset eliminando a estas palabras que no aportan ninguna información a la hora de clasificar los textos. 

Vamos a probar **normalizar** los documentos, y ver cómo cambia:

Para normalizar los documentos, quitamos todos los caracteres que no sean letras del abecedario o espacios, pasamos todo el texto a minúsculas y le quitamos los espacios de inicio y final. Una vez hecho esto, usamos el `WordPunctTokenizer` de `nltk` para separar los elementos por palabras, y hacemos un último filtro en el que quitamos palabras de la lista `stop_words`. Esta lista la hemos obtenido de `nltk` también, pero además hemos añadido `br`, puesto que no ofrece ninguna información semántica.

In [ ]:
wpt = nltk.WordPunctTokenizer()
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english') + ['br']

@np.vectorize
def normalize_corpus(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Procedemos a aplicar la normalización descrita a las reseñas de nuestros datos

In [ ]:
normalized_reviews = normalize_corpus(df.review)

Y repetimos el estudio de las palabras más frecuentes que hicimos antes, pero con los documentos normalizados 

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
doc_word_freq = vectorizer.fit_transform(normalized_reviews)
feature_names = vectorizer.get_feature_names_out()
word_freq = np.squeeze(np.asarray(doc_word_freq.sum(axis=0)))

top_10_freq_words = word_freq.argsort()[::-1][:10]
for top, word in enumerate(top_10_freq_words):
    print(f"Position {top + 1}: {feature_names[word]} with frequency {word_freq[word]}")

Position 1: movie with frequency 83536
Position 2: film with frequency 74478
Position 3: like with frequency 39001
Position 4: good with frequency 28575
Position 5: time with frequency 23276
Position 6: really with frequency 22951
Position 7: story with frequency 22103
Position 8: great with frequency 17821
Position 9: bad with frequency 17720
Position 10: people with frequency 17542


Podemos observar que ahora las palabras más frecuentes son palabras que nos dan más información. Tiene sentido que `movie` y `film` sean las palabras más usadas, puesto que nuestro banco de datos son de reseñas de películas. También tiene sentido que aparezcan palabras como `like`, `good`, `great` o `bad`, pues la gente ha escrito las reseñas con la intención de compartir su opinión de las películas

**De ahora en adelante, usaremos el corpus normalizado**

In [ ]:
df2 = pd.DataFrame({"reviews": normalized_reviews, "sentiment": df.sentiment})
df_sin_normalizar = df
df = df2
df.head()

,reviews,sentiment
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


> Crea una partición de los datos dejando el 80% para entrenamiento y el 20% restante para test usando la función `train_test_split` de sklearn. Comprueba que la distribución de los ejemplos en las clases es la misma en entrenamiento y test. 

Antes de separar los datos en dos conjuntos, vamos a pasar los reviews por un Tokenizer, y convertirlos en `Sequence`. De esta forma se trabaja mejor con Word Embedding.

El `Tokenizer` devuelve una secuencia de palabras, que se corresponde con una lista de enteros, índices de las palabras a un diccionario. De la documentación de Keras:
> This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...

Podemos recuperar el texto dada una sequencia accediendo al diccinario interno del `tokenizer`, llamado `index_word` (Pues vincula índices con palabras, al contrario que `word_inex`, que vincula palabras con índices)

Hemos decidido que nuestras reseñas se quedaran únicamente con `20` comentarios
como máximo. Para que todos los documentos tengan el mismo tamaño, añadimos `0`s a todo documento que se quede con menos elementos usando `pad_sequences`

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = 1500
max_comment_length = 20

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.reviews)

sequences = tokenizer.texts_to_sequences(df.reviews)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
max_words = len(word_index)

# pad_sequences is used to ensure that all sequences in a list have the same length.
data = pad_sequences(sequences, maxlen=max_comment_length)

Found 175616 unique tokens.


Aquí vemos la representación de la primera reseña, con cada palabra siendo representada por un vector de índices al diccionario interno de `tokenizer`.


In [ ]:
data[0]

array([ 378,  505,   14,  141,   13,  632,  696,  540, 1077,  548,  434,
        808, 1077,  441,   56,  100,  301,   14, 1080,  387], dtype=int32)

Para comprobar que de verdad está pasando lo que hemos explicado, recuperamos las palabras asociadas a cada índice

In [ ]:
review_1 = np.array([tokenizer.index_word.get(x) for x in data[0]])
review_1
    

array(['kill', 'order', 'get', 'away', 'well', 'middle', 'class',
       'turned', 'prison', 'due', 'lack', 'street', 'prison',
       'experience', 'watching', 'may', 'become', 'get', 'touch', 'side'],
      dtype='<U10')

Podemos comprobar que, en efecto, se trata de la primera reseña de nuestro set de datos

In [ ]:
df.loc[0,'reviews']

'one reviewers mentioned watching oz episode youll hooked right exactly happened mebr first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordbr called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awaybr would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle cl

Ahora dividimos en 80% de entrenamiento y 20% de prueba.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, df.sentiment, test_size=0.2, train_size=0.8, random_state=RANDOM_STATE)

In [ ]:
# Veamos que tienen la misma distribución
print(f"Número de casos en train set: {len(x_train)}")
print("Casos positivos: ", np.sum(y_train.values))
print("Casos negativos: ", len(y_train) - np.sum(y_train.values))

print(f"Número de casos en test set: {len(x_test)}")
print("Casos positivos: ", np.sum(y_test.values))
print("Casos negativos: ", len(y_test) - np.sum(y_test.values))

Número de casos en train set: 40000
Casos positivos:  19988
Casos negativos:  20012
Número de casos en test set: 10000
Casos positivos:  5012
Casos negativos:  4988


Se puede observar que la distribución de casos positivos y negativos es más o menos igual tanto en train set como en test set.

## 2) Estudio del efecto de distintas configuraciones de word embeddings para resolver la tarea

> Usa distintas configuraciones de word embeddings y discute los resultados obtenidos.

Vamos a comprar el resultado obtenido entre:

- Entrenar desde cero los `Embeddings` con nuestro corpus
- Usar embeddings pre-entrenados, congelando el modelo (sin que aprenda de nuestro corpus)
- Usar embeddings pre-entrenados, sin congelar el modelo

### Modelo 1: Entrenar desde cero con `Embedding` Layer

Primero probamos a crear nuestra propia red neuronal que intenta adivinar los embeddings de nuestro conjunto de entrenamiento. Para ello empleamos las interfaces de `tf.keras`, que usa TensorFlow para construir fácilmente redes neuronales.

In [ ]:
# Fijamos el tamaño de los embedding a 50 dimensiones
embedding_dim = 50

Importamos de `keras.models` el modelo de nuestra red neuronal. Queremos una red neuronal simple, cuyo único propósito sea pasar nuestros documentos (representados como una lista de índices, representando palabras) a la capa `Embedding`, que genera un vector por cada palabra de dimensión `embedding_dim`.

Estos vectores los aplanamos y se los pasamos a una capa densa con la función de activación del sigmoide $\sigma(x) =\frac{1}{1 + e^{-x}}$ que intenta predecir el sentimiento de nuestros documentos. Un `1` implica sentimiento positivo, un `0` sentimiento negativo

In [ ]:
from keras.models import Sequential # Modelo de la red neuronal
from keras.layers import Flatten, Dense, Embedding # Distintas capas que puede tener nuestra red neuronal

In [ ]:
model1 = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_comment_length),
    Flatten(),
    Dense(1, activation='sigmoid') # Esta capa es la que realiza la clasificación en realidad
])
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()
history = model1.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            8780800   
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,781,801
Trainable params: 8,781,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1250/1250 [==============================] - 19s 14ms/step - loss: 0.4845 - accuracy: 0.7692 - val_loss: 0.4380 - val_accuracy: 0.7939
Epoch 2/20
1250/1250 [==============================] - 7s 6ms/step - loss: 0.3937 - accuracy: 0.8211 - val_loss: 0.4421 - val_accuracy: 0.7926
Epoch 3/20
1250/12

In [ ]:
score1 = model1.evaluate(x_test, y_test)

print("Accuracy: %.2f%%" % (score1[1]*100))

313/313 [==============================] - 1s 2ms/step - loss: 2.1340 - accuracy: 0.7238
Accuracy: 72.38%


Podemos observar que embedding ha aprendido accediendo a la salida de las capas intermedias:

In [ ]:
import keras
intermediate_layer_model = keras.models.Model(inputs=model1.input,
                                 outputs=model1.layers[0].output)
embedding_review_1 = intermediate_layer_model.predict(data[0])

1/1 [==============================] - 0s 35ms/step


Ahora tenemos en `embedding_review_1` el embedding que ha sacado nuestro modelo para las 20 palabras de la primera reseña. Observemos, por curiosidad, el embeddin de la primera palabra

In [ ]:
review_1[0], embedding_review_1[0]

('kill',
 array([ 0.49240118, -0.06158967, -0.08537921,  0.01198269,  0.29855233,
        -0.00182005, -0.14153896,  0.11806239, -0.36324027, -0.1639899 ,
         0.12262842,  0.31116223,  0.26826862,  0.10656832,  0.18166742,
         0.12048254, -0.21079075, -0.17963287, -0.00678047, -0.18901734,
         0.22054464, -0.12437028, -0.11108056,  0.22728847, -0.14963979,
        -0.00364058, -0.256805  , -0.14200062,  0.1679555 , -0.5051072 ,
        -0.1786734 , -0.18728876,  0.41131008,  0.07223381, -0.03151915,
         0.07832994,  0.09876098,  0.33823475, -0.121471  , -0.10464939,
        -0.1332437 , -0.2762028 , -0.4143862 ,  0.10521372,  0.19203554,
         0.06034961,  0.34253028, -0.08648186, -0.13975914, -0.18590447],
       dtype=float32))

De alguna manera parece que nuestra red neuronal ha decidido representar la palabra `kill` con este vector de 50 (`embedding_dim`) valores

### Modelo 2: Usar Word Embeddings ya hechos, sin reentrenar

A veces conviene usar Word Embeddings ya entrenados previamente porque tenemos pocos datos para entrenar un modelo bueno (en nuestro caso tenemos 50K datos y no es un problema grande).

Vamos a usar el Word embedding con 50 dimensiones, creado por Stanford y entrenado con 2014 English Wikipedia + Gigaword 5, basado en algoritmo de GloVe.

In [ ]:
import os
import numpy as np

glove_dir = '/content/drive/MyDrive/IA2/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 111252 word vectors.


Creamos la matriz de embedding para insertar en el modelo.

In [ ]:
embedding_dim = 50

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

Creamos un nuevo modelo, pero esta vez con una capa `Embedding` cuyos pesos sobreescribimos con la matriz anterior. Además, ponemos `trainable` a `False` para asegurarnos que al entrenar nuestra red neuronal, los pesos de la capa `Embedding` no cambien

In [ ]:
# MODELO 2. EMBEDDINGS PRE-ENTRENADOS CONGELADOS
model2 = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_comment_length),
    Flatten(),
    Dense(1, activation='sigmoid'),
])
model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 50)            8780800   
                                                                 
 flatten_2 (Flatten)         (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,781,801
Trainable params: 1,001
Non-trainable params: 8,780,800
_________________________________________________________________


In [ ]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model2.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6064 - accuracy: 0.6658 - val_loss: 0.5895 - val_accuracy: 0.6884
Epoch 2/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.5691 - accuracy: 0.7031 - val_loss: 0.5872 - val_accuracy: 0.6921
Epoch 3/20
1250/1250 [==============================] - 3s 3ms/step - loss: 0.5656 - accuracy: 0.7065 - val_loss: 0.5880 - val_accuracy: 0.6883
Epoch 4/20
1250/1250 [==============================] - 3s 3ms/step - loss: 0.5640 - accuracy: 0.7074 - val_loss: 0.5812 - val_accuracy: 0.6949
Epoch 5/20
1250/1250 [==============================] - 4s 4ms/step - loss: 0.5637 - accuracy: 0.7081 - val_loss: 0.5808 - val_accuracy: 0.6956
Epoch 6/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5637 - accuracy: 0.7086 - val_loss: 0.5824 - val_accuracy: 0.6913
Epoch 7/20
1250/1250 [==============================] - 4s 3ms/step - loss: 0.5630 - accuracy: 0.7083 - val_loss: 0.5892 - val_accuracy:

In [ ]:
score2 = model2.evaluate(x_test, y_test)
print("Accuracy: %.2f%%" % (score2[1]*100))

313/313 [==============================] - 1s 3ms/step - loss: 0.5826 - accuracy: 0.6973
Accuracy: 69.73%


### Modelo 3: Usar Word Embeddings ya hechos, reentrenando

Finalemente, probamos a usar la matriz de GloVe que usamos anteriormente, pero sí que permitimos a nuestra red neuronal que cambie los pesos asociados a la capa de `Embedding`. De esta manera, podemos aprender un embedding más especializado, pero partiendo de la base de la matriz anterior.

In [ ]:
# MODELO3. EMBEDDINGS PREENTRENADOS SIN CONGELAR
model3 = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_comment_length),
    Flatten(),
    Dense(1, activation='sigmoid'),
])
model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = True
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 50)            8780800   
                                                                 
 flatten_3 (Flatten)         (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 8,781,801
Trainable params: 8,781,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model3.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/20
1250/1250 [==============================] - 18s 14ms/step - loss: 0.5416 - accuracy: 0.7188 - val_loss: 0.4754 - val_accuracy: 0.7721
Epoch 2/20
1250/1250 [==============================] - 8s 7ms/step - loss: 0.4251 - accuracy: 0.8008 - val_loss: 0.4476 - val_accuracy: 0.7904
Epoch 3/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.3934 - accuracy: 0.8207 - val_loss: 0.4496 - val_accuracy: 0.7930
Epoch 4/20
1250/1250 [==============================] - 7s 6ms/step - loss: 0.3690 - accuracy: 0.8326 - val_loss: 0.4666 - val_accuracy: 0.7839
Epoch 5/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.3435 - accuracy: 0.8492 - val_loss: 0.4743 - val_accuracy: 0.7850
Epoch 6/20
1250/1250 [==============================] - 7s 6ms/step - loss: 0.3166 - accuracy: 0.8642 - val_loss: 0.4987 - val_accuracy: 0.7767
Epoch 7/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.2912 - accuracy: 0.8787 - val_loss: 0.5314 - val_accurac

In [ ]:
score3 = model3.evaluate(x_test, y_test)
print("Accuracy: %.2f%%" % (score3[1]*100))

313/313 [==============================] - 1s 2ms/step - loss: 1.2412 - accuracy: 0.7325
Accuracy: 73.25%


## 3) Análisis final

> Analiza con detalle el mejor clasificador. Busca un ejemplo mal clasificado de cada clase, justifica el error ¿se te ocurre alguna forma de solucionarlo?
> 
> Compara los resultados obtenidos con y sin word embeddings



In [ ]:
print("Sin word embeddings pre-entrenados")
print("Accuracy: %.2f%%" % (score1[1]*100))
print("Con word embeddings pre-entrenados congelados")
print("Accuracy: %.2f%%" % (score2[1]*100))
print("Con word embeddings pre-entrenados sin congelar")
print("Accuracy: %.2f%%" % (score3[1]*100))

Sin word embeddings pre-entrenados
Accuracy: 72.38%
Con word embeddings pre-entrenados congelados
Accuracy: 69.73%
Con word embeddings pre-entrenados sin congelar
Accuracy: 73.25%


El mejor resultado se ha conseguido usando el modelo con Word Embeddings preentrenados y permitiendo que se entrene con nuestros documentos.

El peor resultado es con modelo preentrenado pero sin permitir entrenamiento. 

Esto puede deberse a que nuestro conjunto de entrenamiento ya es suficientemente grande (40K), por lo que no tenemos el problema de falta de datos para entrenar embedding desde 0. Por eso se ha obtenido un resultado mejor partiendo de un embedding desde 0.

In [ ]:
y_test_pred = np.array([1 if probability > 0.5 else 0 for row in model1.predict(x_test) for probability in row])

313/313 [==============================] - 0s 1ms/step


Observamos que hay muchas muestras que no han sido clasificadas correctamente

In [ ]:
sum(y_test_pred != y_test)

2762

In [ ]:
[(np.array([tokenizer.index_word.get(idx) for idx in seq]),pred) 
    for pred, seq in zip(y_test_pred[y_test_pred != y_test.values],x_test[y_test_pred != y_test.values])][:5]

[(array(['masterpiece', 'clearly', 'outstanding', 'well', 'pace', 'slow',
         'cinematography', 'beautiful', 'next', 'time', 'tv', 'youll',
         'never', 'see', 'better', 'tv', 'movie', 'never', 'felt', 'good'],
        dtype='<U14'),
  1),
 (array(['movie', 'would', 'really', 'recommend', 'see', 'apart',
         'probably', 'going', 'like', 'especially', 'lot', 'talking',
         'animals', 'ill', 'give', 'movie', 'rating', 'stars', 'possible',
         'stars'], dtype='<U10'),
  1),
 (array(['likely', 'enough', 'good', 'look', 'favorite', 'dr', 'hours',
         'actually', 'number', 'like', 'evil', 'course', 'look', 'four',
         'laugh', 'youll', 'love', 'scifi', 'ones', 'fantastic'],
        dtype='<U9'),
  1),
 (array(['later', 'place', 'made', 'things', 'may', 'help', 'movie', 'plot',
         'could', 'made', 'interesting', 'question', 'mind', 'movie',
         'ended', 'movie', 'really', 'ended', 'movie', 'ended'],
        dtype='<U11'),
  0),
 (array([None, None

Ya hemos mostrado un par de ejemplos en el que nuestro clasificador ha fallado. Sin embargo, la secuencia de palabras no da mucha información sobre que decía el mensaje, por lo que procedemos a buscar un ejemplo.

En este caso, buscamos una reseña con sentimiento negativo que use la palabra `"masterpiece"`, que probablemente concuerde con la predicción errónea que hemos hecho (pues no imagino que haya muchas reseñas negativas que usen palabras como `"masterpiece"`)

In [ ]:
for num, data in enumerate(zip(df.sentiment,df.reviews)):
    sentiment, review = data
    if 'masterpiece' in review and sentiment == 0:
        print(num)
        print(review)
        break

71
honestly short film sucks dummy used necro scene pretty well made still phony enough looking ruin viewing experience unearthed dvd crisp clear havent made mind helps hinders film little grainy might added creepiness factor going idea film much hype surrounding subject matter honest necrophilia scenes films like nekromantik visitor q among others shocking aftermath talk film loneliness manner deep philosophy bull expensive beautifully filmed turd shocking disgusting insist viewing rent give fact many people make explicit movies necrophilia definitely bigger selection us sickos filming good gore watching rubbery looking doll get cut open considered gore absolutely nothing going overhyped mess hand genesis cerdas sequel aftermath available double feature released unearthed films absolute masterpiece short film really showing good director cerda really given right material although dont care aftermath genesis well made forgive cerda definitely keep eye future


La reseña que encontramos parece concordar con la clasificaión errónea que hemos realizado, pues aunque es negativa sí que incluye todas palabras de la secuencia, que tienen connotaciones positivas.